In [2]:
from dotenv import load_dotenv
load_dotenv(".env.local")
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

/usr/local/python/3.12.1/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:

pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.delete_index(name="rag")
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


In [6]:
import json
data = json.load(open("reviews.json"))
data = ['reviews']

In [7]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })


TypeError: list indices must be integers or slices, not str

In [ ]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)